## Introduction

The aim of this notebook is to be able to create agentic AI system that can pull data from a given URL based on the questions asked by the USER.
And then answer question based on that data. If plots are needed the system should be able to write the python code and show the visulisation


### Test2SQL Q&A Bot on Namma Yatri Data
###

Step 1: The Bot should fetch the data from Namma Yatri for analysis and store it as JSON file


In [41]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
OPEN_API_KEY=''
import openai

# Set your API key
openai.api_key = OPEN_API_KEY

In [186]:
fetch_data_msg='''
You are a Data Enginner who  can fetch data from Namma Yatri Open Data and can generate the code to retrieve and save that data as a dataframe.
the output dataframe you will add column Current Date which will be YYYYmmDDHHMM .The current date shoulkd be same across all the files extracted =.
The name of the files you save it : if you have fetched data from https://d11gklsvr97l1g.cloudfront.net/open/json-data/driver_eda_wards_new_key.json  on 2024 Nov 15, 14:06 hours you 
will save the file as driver_eda_wards_new_key. If the file already exists you will overwrite that file

The urls from which you should fetch the data are:
1. driver_eda_wards_new_key : https://d11gklsvr97l1g.cloudfront.net/open/json-data/driver_eda_wards_new_key.json
2. funnel_cumulative_ward_new_key: https://d11gklsvr97l1g.cloudfront.net/open/json-data/funnel_cumulative_ward_new_key.json
3.funnel_live_ward_new_key:https://d11gklsvr97l1g.cloudfront.net/open/json-data/funnel_live_ward_new_key.json
4.trends_cumulative_ward_new_key: https://d11gklsvr97l1g.cloudfront.net/open/json-data/trends_cumulative_ward_new_key.json
5. trends_live_ward_new_key : https://d11gklsvr97l1g.cloudfront.net/open/json-data/trends_live_ward_new_key.json

Also, while creating the dataframe From ward_num, extract Ward_Number Column which is the number after b_ and add it as a column
Read the file WARDS_ID_MAPPING.xlsx and rename column Ward No. with Ward_Number. Merge every dataframe scraped from Namma Yatri with the WARDS_ID_MAPPING Data on Ward_Number
The output file should be a csv file and before merging ensure in both the files the Ward_Number is string.

You will generate only the python code that can be executed. 
'''

In [108]:
response_format={
    "type": "json_schema",
    "json_schema": {
      "name": "python_code",
      "strict": True,
      "schema": {
        "type": "object",
        "properties": {
          "code": {
            "type": "string",
            "description": "The Python code to be executed."
          },
          "description": {
            "type": "string",
            "description": "A description of what the Python code does."
          },
          "function_name": {
            "type": "string",
            "description": "The name of the function defined in the Python code."
          },
          "parameters": {
            "type": "array",
            "description": "A list of parameters accepted by the function.",
            "items": {
              "type": "string",
              "description": "The name of a parameter."
            }
          },
          "return_type": {
            "type": "string",
            "description": "The type of value returned by the function."
          }
        },
        "required": [
          "code",
          "description",
          "function_name",
          "parameters",
          "return_type"
        ],
        "additionalProperties": False
      }
    }
  }

In [109]:
def generate_completion(user_msg,system_msg, response_format):
  messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_msg
          }
        ]
      },{
          "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_msg
          }]

      }]
  client = openai.OpenAI(api_key=OPEN_API_KEY)
  response = client.chat.completions.create(model="gpt-4o-mini",messages=messages,
                                              temperature=1,max_tokens=4096,top_p=1,
                                              frequency_penalty=0,presence_penalty=0,
                                              response_format=response_format)
  return response.choices[0].message.content

In [210]:
import json
import subprocess 
import sys

def fetchData():
    output=generate_completion(fetch_data_msg,"You are an expert in Python and always import all necessary libraries in the code",response_format)
    code=json.loads(output)['code']
    print(code)
    result=subprocess.run([sys.executable, "-c", code], capture_output=True)
    return result



In [211]:
result=fetchData()

import pandas as pd
import requests
import json
from datetime import datetime

# Function to fetch and save data as a dataframe

def fetch_and_save_data():
    urls = {
        'driver_eda_wards_new_key': 'https://d11gklsvr97l1g.cloudfront.net/open/json-data/driver_eda_wards_new_key.json',
        'funnel_cumulative_ward_new_key': 'https://d11gklsvr97l1g.cloudfront.net/open/json-data/funnel_cumulative_ward_new_key.json',
        'funnel_live_ward_new_key': 'https://d11gklsvr97l1g.cloudfront.net/open/json-data/funnel_live_ward_new_key.json',
        'trends_cumulative_ward_new_key': 'https://d11gklsvr97l1g.cloudfront.net/open/json-data/trends_cumulative_ward_new_key.json',
        'trends_live_ward_new_key': 'https://d11gklsvr97l1g.cloudfront.net/open/json-data/trends_live_ward_new_key.json'
    }
    # Read the WARDS_ID_MAPPING.xlsx file
    wards_mapping = pd.read_excel('WARDS_ID_MAPPING.xlsx')
    wards_mapping.rename(columns={'Ward No.': 'Ward_Number'}, inplace=True)
    wards_mappi

In [243]:
table_definition={"funnel_live_ward_new_key.csv":'''
  \nThis table contains information about aggregated trips details for a given Ward in Bangalore for the current day.
  The schema is as below:
                  
    \nward_num (VARCHAR):\n\nThe identifier for the ward (e.g., b_1, b_2, b_10). Drop off ward_num=w_0 from the dataset

  \nsrch_fr_e (INTEGER):\n\nNumber of searches made by users for fare estimates.
  \nsrch_which_got_e (INTEGER):\n\nNumber of searches that resulted in a fare estimate being provided.\
  \n srch_fr_q (INTEGER):\n\nNumber of searches made by users for trip quotes.

  \nsrch_which_got_q (INTEGER):\n\nNumber of searches that resulted in a trip quote being provided.
  \nbooking (INTEGER):\n\nThe number of bookings made from the searches.
  \nongng_ride (FLOAT):\n\nNumber of ongoing rides at the time of data collection.
  \ndone_ride (INTEGER):\n\nNumber of completed rides.
  \ncancel_ride (INTEGER):\n\nNumber of rides that have been canceled.
  \navg_dist_pr_trip (FLOAT):\n\nThe average distance per trip.

  \navg_fare (FLOAT):\n\nThe average fare per ride.\n
  earning (INTEGER):\n\nTotal earnings (in currency) from trips.
  \ndist (FLOAT):\n\nTotal distance covered by drivers for the trips.
  \nsrch_to_e_rate (FLOAT):\n\nConversion rate from searches to fare estimate.
  \ne_to_q_srch_rate (FLOAT):\n\nConversion rate from fare estimate to trip quote search.
  \nq_accept_rate (FLOAT):\n\nThe rate at which quotes are accepted.
  \nq_to_bkng_rate (FLOAT):\n\nConversion rate from quotes to bookings.
  \nbkng_cancel_rate (FLOAT):\n\nCancellation rate of bookings.
  \ncnvr_rate (FLOAT):\n\nOverall conversion rate from searches to successful bookings.''',
  
  
  "driver_eda_wards_new_key.csv":'''This table contains the data on driver activities across different wards.
  \n\nward_num (VARCHAR): The ward identifier (e.g., "b_1", "b_2"). Drop off ward_num w_0 from the data
  \nactive (BOOLEAN): Whether the ward is active or not.
  \nactive_drvr (INTEGER): The number of online drivers in the ward.
  \ndrvr_notonride (INTEGER): The number of  online drivers not currently on a ride.
  \ndrvr_onride (INTEGER): The number of online drivers currently on a ride.
  \nZone (VARCHAR): The zone the ward belongs to
  \nAC Name (VARCHAR): The Assembly Consituency Name with ID.Multiple wards belong to an Assembly constituency
  \nWard No and Name (VARCHAR): The ward no-ward name
  \nAC_ID (INTEGER): Assembly Consituency ID
  \ASSEMBLY CONSTITUENCY (VARCHAR): Assembly Constituency Name.
  \nWard_Number (VARCHAR): Ward number without the b_ in ward_num
  ''',

  "funnel_cumulative_ward_new_key.csv":'''
  \nThis table contains information about aggregated trips details for a given Ward in Bangalore since 2022 .
  The schema is as below:
                  
    \n ward_num (VARCHAR):\n\nThe identifier for the ward (e.g., b_1, b_2, b_10). Drop off ward_num=w_0 from the dataset

  \nsrch_fr_e (INTEGER):\n\nNumber of searches made by users for fare estimates.
  \nsrch_which_got_e (INTEGER):\n\nNumber of searches that resulted in a fare estimate being provided.\
  \n srch_fr_q (INTEGER):\n\nNumber of searches made by users for trip quotes.

  \nsrch_which_got_q (INTEGER):\n\nNumber of searches that resulted in a trip quote being provided.
  \nbooking (INTEGER):\n\nThe number of bookings made from the searches.
  \nongng_ride (FLOAT):\n\nNumber of ongoing rides at the time of data collection.
  \ndone_ride (INTEGER):\n\nNumber of completed rides.
  \ncancel_ride (INTEGER):\n\nNumber of rides that have been canceled.
  \navg_dist_pr_trip (FLOAT):\n\nThe average distance per trip.

  \navg_fare (FLOAT):\n\nThe average fare per ride.\n
  earning (INTEGER):\n\nTotal earnings (in currency) from trips.
  \ndist (FLOAT):\n\nTotal distance covered by drivers for the trips.
  \nsrch_to_e_rate (FLOAT):\n\nConversion rate from searches to fare estimate.
  \ne_to_q_srch_rate (FLOAT):\n\nConversion rate from fare estimate to trip quote search.
  \nq_accept_rate (FLOAT):\n\nThe rate at which quotes are accepted.
  \nq_to_bkng_rate (FLOAT):\n\nConversion rate from quotes to bookings.
  \nbkng_cancel_rate (FLOAT):\n\nCancellation rate of bookings.
  \ncnvr_rate (FLOAT):\n\nOverall conversion rate from searches to successful bookings.
  \nZone (VARCHAR): The zone the ward belongs to
  \nAC Name (VARCHAR): The Assembly Consituency Name with ID.Multiple wards belong to an Assembly constituency
  \nWard No and Name (VARCHAR): The ward no-ward name
  \nAC_ID (INTEGER): Assembly Consituency ID
  \ASSEMBLY CONSTITUENCY (VARCHAR): Assembly Constituency Name.
  \nWard_Number (VARCHAR): Ward number without the b_ in ward_num
  
  ''',

}

In [246]:
### Step 2: get the latest fetched data and answer the questions

system_msg=f'''
You are a datascientist and your job is to write python code that can execute without any error,
to read from the correct csv files. While answering always use the data of the max of the Current Date 
data present. 

Any graph you generate should use plotly/plotly express so that it can be interactive. The csv file and the schema definitions are as below:

{str(table_definition)}

Ignore Rows where Ward No and Name is Null 

Also, note if needed to merge table to answer a question then merge the tables on column "Ward No and Name" & "AC Name" - consider this to be the key column for all tables

When the user asks information based on WARD NAME then do a like match with rows that contain the WARD NAME and retrieve relevant information. 

The user may not enter the right name always - for example, if I search with Sanjay Nagar or Sanjaya Nagara or Sanjay Nagar the information related to the closest match  is to be retrieved.

When giving ward details always give the Ward No and Name.

In case the output is not a graph - print the response to the standard output
'''

In [248]:
import json
output=generate_completion("What is the Driver Earnings  and how many online drivers are there in Shantala today",system_msg,response_format)
code=json.loads(output)['code']
exec(code)

Driver Earnings in Shantala: 450242
Number of online drivers in Shantala: 540


In [249]:
print(code)

import pandas as pd
import plotly.express as px

# Load the CSV files
funnel_live_ward_df = pd.read_csv('funnel_live_ward_new_key.csv')
driver_eda_wards_df = pd.read_csv('driver_eda_wards_new_key.csv')

# Filter relevant data for matching ward names
ward_name = 'Shantala'
filtered_driver_data = driver_eda_wards_df[driver_eda_wards_df['Ward No and Name'].str.contains(ward_name, case=False, na=False)]
filtered_funnel_data = funnel_live_ward_df[funnel_live_ward_df['Ward No and Name'].str.contains(ward_name, case=False, na=False)]

# Merge dataframes on 'Ward No and Name' & 'AC Name'
merged_data = pd.merge(filtered_driver_data, filtered_funnel_data, on=['Ward No and Name', 'AC Name'])

# Get Driver Earnings and Active Drivers
earnings = merged_data['earning'].sum()
active_drivers = merged_data['active_drvr'].sum()

# Print the result
print(f'Driver Earnings in {ward_name}: {earnings}')
print(f'Number of online drivers in {ward_name}: {active_drivers}')


In [250]:
import json
output=generate_completion("Which is the Ward Name where driver earnings are more today?",system_msg,response_format)
code=json.loads(output)['code']
exec(code)

Ward Name: 181-Shantala Nagar, Earnings: 450242


In [251]:
import json
output=generate_completion("Top 5 Wards which has highest average fare per ride as table today",system_msg,response_format)
code=json.loads(output)['code']
exec(code)

      Ward No and Name  avg_fare
15        112-Varthuru  230.9005
16    113-Munnekollala  209.8508
143  228-Naganathapura  208.4484
199           6-Kogilu  204.3743
17    114-Marathahalli  202.7754


In [252]:
import json
output=generate_completion("Top 10 Wards which has highest cancellation rate cumulative as a bar graph",system_msg,response_format)
code=json.loads(output)['code']
exec(code)

In [253]:
output=generate_completion("Total Searches for Quotes by Ward Name in Hebbal Assembly Constituency sort in descending order of searches",system_msg,response_format)
code=json.loads(output)['code']
exec(code)

In [254]:
print(code)

import pandas as pd
import plotly.express as px

# Read the data from CSV files
funnel_live = pd.read_csv('funnel_live_ward_new_key.csv')
driver_eda = pd.read_csv('driver_eda_wards_new_key.csv')
funnel_cumulative = pd.read_csv('funnel_cumulative_ward_new_key.csv')

# Drop rows where 'Ward No and Name' is null
funnel_live = funnel_live[funnel_live['Ward No and Name'].notnull()]
driver_eda = driver_eda[driver_eda['Ward No and Name'].notnull()]
funnel_cumulative = funnel_cumulative[funnel_cumulative['Ward No and Name'].notnull()]

# Merging dataframes on 'Ward No and Name' & 'AC Name'
merged_live = funnel_live.merge(driver_eda, on=['Ward No and Name', 'AC Name'], how='inner')
merged_cumulative = funnel_cumulative.merge(driver_eda, on=['Ward No and Name', 'AC Name'], how='inner')

# Filtering to only include records from Hebbal Assembly Constituency
hebbal_data = merged_cumulative[merged_cumulative['AC Name'].str.contains('Hebbal', case=False)]

# Sorting by the number of searches for quot